In [1]:
import pandas as pd
from oracle import oracleTopd

data = oracleTopd('select * from recipe_data_table where rownum <= 1000') # 데이터 불러오기 (1000개, 15초)

ModuleNotFoundError: No module named 'oracle'

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv(r'C:\Users\HwijunKwon\github\recipe\models\data\RecipeData.csv')
data = data.loc[:1000]

In [5]:
import random 
import numpy as np

np.random.seed(77)
toy = data[["recipe_title", "recipe_step", "recipe_ingredients"]] # toy 데이터 만들기 표본 1000개
toy.head()

,recipe_title,recipe_step,recipe_ingredients
0,비빔밥재료 애호박나물 애호박볶음 아삭거리게 볶는 방법,"['#1 호박은 0.5cm 정도 두께로 자른 후 편하게 채썰어주세요.', '#2 너...","{'재료': ['애호박1개', '다진마늘0.5큰술', '소금0.5큰술', '참기름1..."
1,[마늘종볶음]단짠단짠 자꾸만 손이가요~,"['#1 마늘종 300g을 4cm길이로 썰어주세요', '#2 달궈진 마른팬에 건새우...","{'재료': ['건새우50g', '물1/4컵', '마늘종300g', '소금1/3큰술..."
2,백종원의 부추 달걀 볶음,"['#1 부추는 적당한 크기로 썰고 달걀은 여러번 저어서 곱게 풀어 놓습니다', '...","{'재료': ['부추1줌', '달걀4개', '식용유2큰술', '밥2공기'], '양념..."
3,간단하게 맛난 반찬 만들기 '베이컨 버섯볶음',"['#1 버섯과 양파를 잘라줍니다.', '#2 올리브유를 두르고 양파와 다진마늘을 ...","{'재료': ['베이컨5~6줄', '양파1/2개', '미니새송이버섯7개', '다진마..."
4,고소하고 향긋한 깻잎들깨볶음,['#1 깻잎 을 씻어줍니다.저는 자잘한 깻잎순으로 했지만. 그냥 큰 잎을 잘라서 ...,"{'재료': ['깻잎', '대파', '다진마늘1T', '국간장1T', '참기름1T'..."


In [6]:
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
import re


# kkma = Kkma()
# komoran = Komoran()
# hannanum = Hannanum()

In [7]:
okt = Okt()
tagged_corpus_list = []
cleaned_tagged_corpus_list = []

for index, row in tqdm(toy.iterrows(), total=len(toy)):
    cleaned_text = re.sub(r'[^A-Za-z0-9가-힣\s]+', '', str(row['recipe_step']))
    words = okt.morphs(cleaned_text)
    tag = row['recipe_title']

    # recipe_step의 길이가 0이 아니면 다음 작업을 수행합니다.
    if len(words) != 0:
        tagged_corpus_list.append(TaggedDocument(words=words, tags=[tag]))


100%|██████████| 1001/1001 [00:33<00:00, 29.60it/s]


In [8]:
from gensim.models import doc2vec

model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)
model.build_vocab(tagged_corpus_list) # Vocabulary 빌드

print(f"Tag Size: {len(model.dv.key_to_index)}", end=' / ') # 수정된 부분: doctags 대신 key_to_index 사용

model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=10) # Doc2Vec 학습
model.save('recipe.doc2vec') # 모델 저장


Tag Size: 972 / 

In [9]:
tagged_corpus_list[:3]

[TaggedDocument(words=['1', '호박', '은', '05', 'cm', '정도', '두께', '로', '자른', '후', '편하게', '채', '썰어주세요', '2', '너무', '얇게', '자르면', '볶은', '후', '물러질', '수', '있어요', '적당히', '도톰하게', '잘라주세요', '3', '팬', '에', '오일', '이나', '들기름', '을', '살짝', '두른', '후', '다진', '마늘', '05', '큰', '술', '과', '소금', '05', '큰', '술', '을', '넣고', '센', '불', '에서', '빠르게', '볶아', '줍니다', '센', '불', '에서', '빠르게', '볶아줘야', '물기', '가', '생기지', '않아요', '4', '23분', '정도', '빠르게', '볶은', '후', '불', '을', '끄고', '참기름', '살짝', '둘러주면', '완성', '입니다', '5', '너무', '오래', '볶으면', '금방', '물러지고', '물이', '생겨요', '센', '불', '에서', '빠르게', '볶아줘야', '호박', '의', '색감', '도', '예쁘고', '식', '감도', '좋아요'], tags=['비빔밥재료 애호박나물 애호박볶음 아삭거리게 볶는 방법']),
 TaggedDocument(words=['1', '마늘', '종', '300', 'g', '을', '4', 'cm', '길이', '로', '썰어주세요', '2', '달궈진', '마른', '팬', '에', '건', '새우', '50', 'g', '을', '넣고', '중간', '불', '에서', '1분', '간', '볶아주세요', '마른', '팬', '에서', '건', '새우', '를', '볶으면', '잡내는', '사라지고', '단맛', '은', '올라갑니다', '3', '볶은', '새우', '를', '체', '에', '올려', '잔', '가루', '를', '털어', '내주세요', '4', '팬', '에', '불', '을'

In [ ]:
model.dv

In [13]:
list(model.dv.key_to_index.keys())

['비빔밥재료 애호박나물 애호박볶음 아삭거리게 볶는 방법',
 '[마늘종볶음]단짠단짠 자꾸만 손이가요~',
 '백종원의 부추 달걀 볶음',
 "간단하게 맛난 반찬 만들기 '베이컨 버섯볶음'",
 '고소하고 향긋한 깻잎들깨볶음',
 '【간단반찬】꽈리고추 햄볶음 만들기~',
 '【스팸감자볶음만들기】',
 '비엔나 소세지강정 간단하게 만들어 먹기 좋아요 ♪',
 '초간단 밑반찬 파프리카버섯볶음 굴소스가 포인트',
 '갈릭마요새우 간단히 만들어 먹는 술안주!',
 "일본식 우엉,당근 볶음 '킨피라'",
 '영양 많고 맛있는 여름반찬 만들기-가지볶음',
 '자취생 간단요리<메추리알볶음>',
 '소고기두부볶음 - 아이들 반찬/이유식 반찬으로도 최고죠~옛생각이 절로...',
 '달걀(계란) 숙주볶음 : 초간단 반찬 겸 술안주',
 '#간단한반찬, 고소함 터지는 들깨두부볶음',
 '간단 반찬-아이 반찬으로 좋은 감자볶음',
 '하찮은 냉동미니돈까스에서 고오급 난자완스 만들기',
 '두부고기 맛이 나는 두부볶음~~',
 '브로콜리들깨볶음, 기분 좋은 고소한 맛',
 '느타리버섯 볶음! 매콤한 버섯요리.',
 '햄 숙주볶음 : 초간단 밥반찬&술안주',
 "볶음밥, 주먹밥 어디든 잘 어울리는 '만능 소고기볶음'",
 '진미채 볶음, 짭쪼름 달달 간장소스, 초간단 밑반찬',
 '아삭한 식감이 매력적인 단짠 반찬 마늘종 볶음',
 '매콤 새송이버섯볶음',
 '호박참치볶음(담백하니 아기도 좋아해요~)',
 '시금치요리 시금치베이컨볶음 엄지척',
 '아삭아삭 달큰한 양배추 베이컨볶음~~',
 '입맛 돋는 양파볶음 만드는법',
 '참치 맛 가루 (후리 가케)',
 '깐풍기맛 나는 깐풍두부!',
 '아삭아삭 식감이 살아있는 소고기오이볶음(아이도 먹기 좋은 밑반찬)',
 '쫄깃한 두부강정 만들기',
 '느타리버섯두루치기',
 '무볶음 만들기',
 '돼지고기 다짐 육 간장 볶음~',
 '호박베이컨볶음',
 '쥐포볶음 만드는법 간단한 밑반찬, 술안주',
 '고추장 건새우볶음 : 밑반찬 하나로 공

In [15]:
model

In [14]:
similar_doc = model.dv.most_similar('비빔밥재료 애호박나물 애호박볶음 아삭거리게 볶는 방법')
print(similar_doc)

[('아기반찬 느타리버섯볶음', 0.8797067403793335), ('시금치베이컨볶음, 고급스러운 혼밥 반찬', 0.8537321090698242), ('소화잘되는 겨울보양식, "무나물볶음"♪', 0.851722776889801), ('초간편 호박볶음', 0.8465582728385925), ('꼬들꼬들 미역줄기볶음', 0.8394210934638977), ('애호박새우젓볶음', 0.8328395485877991), ('큰둥근애호박새우젓볶음!', 0.8286600112915039), ('브로콜리들깨볶음, 기분 좋은 고소한 맛', 0.8261022567749023), ("당도최고 겨울무로 간단히 볶아 만든 '무나물'", 0.8242119550704956), ('[5분 반찬] 초간단 애호박새우젓볶음 : 누구나 좋아하는 감칠맛~', 0.8217670917510986)]


In [92]:
similar_doc_2 = model.dv.most_similar('대패삼겹살튀김 독특한 요리')
print(similar_doc_2)

[('가지튀김/백종원레시피', 0.8012381792068481), ('고구마롤 돈까스', 0.7966592907905579), ('해피투게더 야간매점 이민우 마마밥', 0.7921081781387329), ('든든한 영양간식~ 달걀돈까스', 0.7920870780944824), ('진짜 오징어 튀김을 상큼한 소스와 함께! 크리미어니언오징어링', 0.7861824035644531), ('탕수육', 0.7842554450035095), ('닭가슴살 치킨까스', 0.7822151780128479), ('닭가슴살 치킨까스 만들기', 0.7818322777748108), ('술안주로 딱 좋은 스위트콘전 레시피!', 0.7778521776199341), ('겉바속촉 애호박새우튀김!', 0.7778517007827759)]


### 해결해야할 과제


1. 레시피에서 하는 요리에 대한 정의가 필요함
  - 레시피 타이틀 전처리
  - 레시피 태그 전처리 
2. 레시피에 따라서 2인분, 3인분, 4인분 다양함

In [65]:
data["recipe_summary1"].value_counts()

2인분       57219
4인분       31408
1인분       25937
3인분       19687
6인분 이상    16377
5인분        3670
Name: recipe_summary1, dtype: int64

### 영양소를 고려한 임베딩

In [177]:
toy = data[["recipe_title", "recipe_ingredients"]].iloc[:1000].copy()

In [178]:
# 식재료 칼럼 만들기 (20개)
for i in range(1, 21):
    toy.loc[:, f'ingredient{i}'] = None
    toy.loc[:, f'quantity{i}'] = None
    toy.loc[:, f'unit{i}'] = None


In [191]:
import tqdm
import re
import ast
non_matching_items =[]
for idx, row in tqdm.tqdm(toy.iterrows(), total=toy.shape[0]):
    ingredients_dict = ast.literal_eval(row["recipe_ingredients"])
    ingredient_count = 1
    for category, items in ingredients_dict.items():
        if items:  # 아이템이 존재하는 경우
            for item in items:
                # 숫자가 포함된 경우와 숫자가 없는 경우 모두 처리
                match = re.match(r'([가-힣]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item)
                if match:
                    ingredient, _, quantity, unit = match.groups()
                    
                    toy.at[idx, f'ingredient{ingredient_count}'] = ingredient
                    toy.at[idx, f'quantity{ingredient_count}'] = quantity if quantity else ""  # 숫자가 없는 경우 빈 문자열 할당
                    toy.at[idx, f'unit{ingredient_count}'] = unit if unit else ""  # 단위가 없는 경우 빈 문자열 할당

                    ingredient_count += 1
                else:
                    non_matching_items[idx] = item

for idx, item in non_matching_items.items():
    print(f'Row {idx}: {item}')



100%|██████████| 1000/1000 [00:03<00:00, 254.42it/s]

Row 34: ​고추가루1/2큰술
Row 258: (소주or맛술)1큰술
Row 302: ﻿대파조금
Row 367: 18배멸치육수2T
Row 424: ​간장2수저
Row 598: 
Row 837: 
Row 892: 
Row 895: ​식용유1큰술
